## Python para Finanças - Detecção de Fraudes

Importando a base de dados com transações realizadas por clientes em seus cartões de crédito e identificadas se são ou não fraudulentas para criar o nosso modelo de detecção de fraudes.

In [13]:


# bibliotecas usadas
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [14]:
df = pd.read_csv('https://raw.githubusercontent.com/afonsosr2/python-financas/main/Video_02/df_balanceado.csv')
df.head()

,gender,city,state,job,category,amt,dia,mes,ano,hora,minuto,is_fraud
0,F,Calhan,CO,Sports development officer,shopping_pos,7.42,22,8,2023,1,17,0.0
1,M,Allison Park,PA,Software engineer,misc_net,74.56,20,6,2023,5,11,0.0
2,F,Bethel,OH,"Teacher, secondary school",home,98.78,9,12,2023,15,11,0.0
3,M,Allison Park,PA,Software engineer,misc_pos,17.03,14,7,2023,21,52,0.0
4,M,Staunton,IL,Editorial assistant,home,45.22,12,12,2023,19,22,0.0


Possuímos 12 colunas, sendo 11 delas as características que vamos utilizar para classificar as transações como **genuínas** ou **fraudulentas** e uma delas é a variável resposta ou alvo.

Dicionário de dados:

- gender - sexo biológico do titular do cartão de crédito
- city - Cidade do titular do cartão de crédito
- state - Estado do titular do cartão de crédito
- job - Profissão do titular do cartão de crédito
- category - Categoria do comerciante
- amt - Valor da transação
- dia - dia da transação
- mes - mês da transação
- ano - ano da transação
- hora - hora da transação
- minuto - minuto da transação
- is_fraud - Indicador de fraude **(Classe Alvo)**

In [15]:
# separando as features e a variável resposta
X = df.drop(['is_fraud'], axis=1)
y = df['is_fraud']

In [16]:
# dividindo os dados em bases de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

Separando as colunas categóricas que serão aplicadas a transformação dos dados

In [17]:
colunas_categoricas = ['gender', 'city', 'state', 'category']
colunas_numericas = ['amt', 'dia', 'mes', 'ano', 'hora', 'minuto']

cat = OneHotEncoder(sparse_output=True, handle_unknown='ignore')
num = StandardScaler(with_mean=False)

In [18]:
# Criando o ColumnTransformer para aplicar os transformadores
col_transformer = ColumnTransformer( transformers=[
        ('cat', cat, colunas_categoricas),
        ('num', num, colunas_numericas)
    ])

# Criando um pipeline com o pré-processador e o modelo de regressão logística
model = Pipeline(steps=[
    ('col_transformer', col_transformer),
    ('classificação', LogisticRegression(random_state=42))
])

In [19]:
model.fit(X_train, y_train)

m:\apps no sd\Python\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('col_transformer',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['gender', 'city', 'state',
                                                   'category']),
                                                 ('num',
                                                  StandardScaler(with_mean=False),
                                                  ['amt', 'dia', 'mes', 'ano',
                                                   'hora', 'minuto'])])),
                ('classificação', LogisticRegression(random_state=42))])

### Testando o modelo

In [20]:

# Gerando previsões com os dados de teste
y_pred= model.predict(X_test)

# Exibindo a acurácia do modelo
print(accuracy_score(y_test, y_pred))

0.8403141361256544



O nosso modelo tem a acurácia de aproximadamente 85%. Ou seja, A cada 100 transações ele acerta por volta de 85.

Vamos testar com uma transação fraudulenta para observar se ele consegue acertar?

In [44]:

fraude = pd.DataFrame({'gender': ['M'], 'city': ['Santa Ana'], 'state': ['CA'], 'job': ['Designer, textile'],
                       'category': ['grocery_pos'], 'amt': [127.61], 'dia': [1], 'mes': [2], 'ano': [2024], 'hora': [3],
                       'minuto': [37]})
fraude

,gender,city,state,job,category,amt,dia,mes,ano,hora,minuto
0,M,Santa Ana,CA,"Designer, textile",grocery_pos,127.61,1,2,2024,3,37


In [45]:

fraude_pred = model.predict(fraude)[0]
print(fraude_pred)

1.0
